
## About this project

We want a csv with data from different real-state agencies, additionally we need to keep a good architechture to mantain this project over time, because, maybe we will need create a job looking this data automatically.

Technologies used:
- Selenium
- BeautifulSoup
- Pandas

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from selenium import webdriver
import pandas as pd
import time
import random
import sys
import os
from datetime import datetime
sys.path.append(os.path.abspath('../'))
from utils import (soup_content_from_url,
                    elements_from_soup_bestsellers_web, 
                    subcategories_from_bestsellers_web,
                    detail_dict_from_product_page,
                    load_obj,
                    save_soup)
from property_utils import InmoclickSearchPage, PropertyType, Property

In [3]:
driver = webdriver.Firefox()

In [4]:
base_url = 'https://www.inmoclick.com.ar'
houses_list = "/inmuebles/venta/lotes-y-terrenos/mendoza"
#houses_list = "/inmuebles/venta/casas/mendoza"
params = "?favoritos=0&limit=48&prevEstadoMap=&provincias=21&precio%5Bmin%5D=&precio%5Bmax%5D=&moneda=1&sup_cubierta%5Bmin%5D=&sup_cubierta%5Bmax%5D=&sup_total%5Bmin%5D=&sup_total%5Bmax%5D=&page="
url = base_url+houses_list+(params+str(1))
url

'https://www.inmoclick.com.ar/inmuebles/venta/lotes-y-terrenos/mendoza?favoritos=0&limit=48&prevEstadoMap=&provincias=21&precio%5Bmin%5D=&precio%5Bmax%5D=&moneda=1&sup_cubierta%5Bmin%5D=&sup_cubierta%5Bmax%5D=&sup_total%5Bmin%5D=&sup_total%5Bmax%5D=&page=1'

### Getting all pages related to our search
We want to get all of HTML pages in memory to preprocess them and use the data asociated. In this first jupyter cell we just get the first page in our search, and with that we will get the max page number to iterate looking for all data in next cells.

In [15]:
inmoclick_search_pages = []
soup = soup_content_from_url(driver, url)
isp = InmoclickSearchPage(soup, PropertyType.LAND)
inmoclick_search_pages.append(isp)
len(isp.search_items())

48

In [ ]:
from urllib.parse import urlparse
import random

pages_range = range(2,isp.max_page_number())
for p in pages_range:
    url = base_url+houses_list+(params + str(p))
    print(url)
    
    soup = soup_content_from_url(driver, url)
    isp = InmoclickSearchPage(soup, PropertyType.LAND)
    inmoclick_search_pages.append(isp)
    
    time.sleep(random.randint(1,5))

print(isp.max_page_number())
print(len(inmoclick_search_pages))

### Casting inmoclick objects to our own object "Property"
We want to create our object because if we add a new agency, different to inmoclick, we will add some new code and we want to stay working inmoclick functions. Additionally we want to treat different properties from different agencies in just one way.

In [107]:
recent_id = 0
properties = []
data = []
for isp in inmoclick_search_pages:
    for item in isp.search_items():
        #isi = InmoclickSearchItem(html_article, PropertyType.LAND)
        recent_id = recent_id + 1
        property_dict = item.to_dict()
        property_dict['url'] = base_url + property_dict['url']
        property_dict['recent_id'] = recent_id
        property_dict['source_web'] = base_url
        p = Property(property_dict)
        properties.append(p)
        data.append(p.to_list())
print(data[0])

['inmueble-85193-2',
 'Las Heras',
 'Las Heras',
 'Mendoza',
 'US$',
 23000.0,
 'US$ 23.000',
 '/19352-carballo-inmobiliaria/inmuebles/784/ficha/lote-en-venta-en-regalado-olguin-s-n?btid=2607340/85193-helga-petek/inmuebles/2/ficha/lote-en-venta-en-las-heras',
 'Terro en calle Perón de Las Heras Mendoza se escucha oferta contado .escucho oferta en pesos a dólar blue.',
 <PropertyType.LAND: 'land'>,
 None,
 '/19352-carballo-inmobiliaria/inmuebles/784/ficha/lote-en-venta-en-regalado-olguin-s-n?btid=2607340',
 1,
 datetime.datetime(2020, 5, 29, 21, 10, 3, 695962),
 '365',
 None,
 None,
 None,
 None,
 True,
 True,
 True]

### Casting our objects to DataFrame
To make an analysis we need our data in a DataFrame object

In [104]:
df = pd.DataFrame(data,columns=Property.attributes_order())
df.head(3)

,ref_id,neighborhood,district,province,currency,amount,price,url,description,property_type,...,recent_id,scrapped_date,totalArea,floorArea,bedrooms,bathrooms,garage,has_gas,has_water,has_electricity
0,inmueble-85193-2,Las Heras,Las Heras,Mendoza,US$,23000.0,US$ 23.000,/19352-carballo-inmobiliaria/inmuebles/784/fic...,Terro en calle Perón de Las Heras Mendoza se e...,PropertyType.LAND,...,1,2020-05-29 19:06:13.881371,365,None,None,None,None,True,True,True
1,inmueble-71779-71,Primitivo de la Reta y Alsina,Godoy Cruz,Mendoza,US$,560000.0,US$ 560.000,/19352-carballo-inmobiliaria/inmuebles/784/fic...,GREENWOOD VENDE EN GODOY CRUZ ESPECTACULAR TER...,PropertyType.LAND,...,2,2020-05-29 19:06:13.884346,2800,None,None,None,None,True,True,True
2,inmueble-71779-65,Pinares de Furlotti,Maipu,Mendoza,US$,20000.0,US$ 20.000,/19352-carballo-inmobiliaria/inmuebles/784/fic...,GREENWOOD VENDE OPORTUNIDAD INCREIBLE\n\nPINAR...,PropertyType.LAND,...,3,2020-05-29 19:06:13.886947,350,None,None,None,None,True,True,True


In [105]:
df.to_csv('out-venta-lotes-y-terrenos.csv',index=False)

In [66]:
df.groupby('currency').count()['ref_id']

currency
$       371
US$    1354
Name: ref_id, dtype: int64

In [68]:
df.has_gas.sum()

1362

In [84]:
df.totalArea[~df.totalArea.str.isnumeric()]

29        10.000
68        16.500
109      1857.26
230      464.51 
232      508.31 
          ...   
2029    19769.27
2037     1105.15
2063      298.39
2075      508.94
2084      628.16
Name: totalArea, Length: 111, dtype: object

In [87]:
df.totalArea = pd.to_numeric(df.totalArea, errors='coerce')